# Introduction

In Chapter 1 we devised a signature for a graph and saw that it works fine with a graph up to order 3.

We saw Signature v1 was a bit too dumb to reach order 4.

In this chapter we are going to write an algorithm that aim at higher orders.



# Definitions

Now that we are warming let's add a few more definitions that I did not mention in the previous chapter.

| Term        | Definition   |
| :---------- | :----------- |
| **Edge** | An edge (or link) is a connection between two nodes in a graph. |
| **Connected Graph** |	A **connected graph** is one where you can travel from any node to any other node by following the edges. There are no isolated "islands" of nodes. |
| **Disconnected Graph** |	A **disconnected graph** is the opposite of a connected graph. The opposite of a connected graph is a disconnected graph. The graph is made up of two or more separate "pieces" or "components" that are not linked to each other by any edges. |
| **Isolated Node** |	An **isolated node** (or isolated vertex) is a node that has no edges connected to it. It has zero neighbours. A graph where all nodes are isolated is sometimes called a "null graph" or "empty graph". |
| **Complete Graph** | A **complete graph** is a graph where every single node is directly connected to every other single node. They are "all connected" to each other in the most direct way possible. |
| **Clique**|	A **clique** in a graph is a group of nodes where every node in that group is directly connected to every other node in that same group. It's like a "mini complete graph" within a larger graph. |

# Better signature

Our next step, Signature v2, builds directly on what we learned from Signature v1.

Remember, Signature v1 simply listed each node's neighbor count. While this was quick and somewhat effective, we saw its limitations pretty quickly.

Signature v2 goes a bit deeper: for each node, it will not only list its own neighbor count, but also the neighbor counts of its neighbors. This gives us a richer picture of the local connections around each node.





The primary goal of the signature v2 algorithm is to generate a deterministic and unique signature for a graph.
The graph signature is an array of a node signature for each node of the graph.

The process is iterative. It starts with a simple characterization of each node and recursively refines the signatures of ambiguous nodes until all possible distinctions have been made.

The Example Graph
Throughout this document, we will use the following 6-node graph as a reference to illustrate the algorithm's steps.


![fig1: Simple Graph](images/simple-graph_G1.png)

## Introduction

The goal of this algorithm is to generate a deterministic and unique signature for each node within a graph. This signature, reflecting the node's topological role, is invaluable for tasks like graph isomorphism testing or identifying structurally equivalent nodes. The process is iterative, progressively refining the signatures until all ambiguities are resolved.

## The Example Graph

Throughout this document, we will use the following 6-node graph as a reference.

| Node | Neighbour count | Neighbours |
| :--- | :-------------- | :--------- |
| A    | 3               | B C D      |
| B    | 4               | A D E F    |
| C    | 1               | A          |
| D    | 2               | A B        |
| E    | 2               | B F        |
| F    | 2               | B E        |

## Data Structures

The algorithm operates on two main data structures: the `Node Signature` for each node in the graph, and simple `Neighbor Representation` objects used during the expansion process.

### The Node Signature

This is the primary object representing a node during computation. A signature is considered **`COLLAPSED`** initially. It becomes **`EXPANDED`** when its `neighbors` array is populated. It is **`FINALIZED`** once it is assigned a `finalIndex`.

A `Node Signature` has the following properties:

* **`label`**: `string` - The node's unique identifier (e.g., 'A'). Used for tracking.
* **`neighborCount`**: `number` - The degree of the node. This is the primary sorting criterion.
* **`finalIndex`**: `number` (optional) - The node's unique, zero-based rank in the sorted list. A signature is considered `FINALIZED` once this is assigned.
* **`finishStep`**: `number` (optional) - The pass number in which the `finalIndex` was assigned.
* **`neighbors`**: `array` (optional) - If a signature is ambiguous, this array is populated with `Neighbor Representation` objects to resolve the ambiguity.
* **cycleDistance**: number (optional) - A marker used to describe a loop.

### Neighbor Representation

When a `Node Signature` is expanded, its `neighbors` array is populated with node signatures:

* **For a finalized neighbor**: `{ finalIndex: number }`
    * A reference to a neighbor that is already unambiguous.
* **For a non-finalized neighbor**: `{ neighborCount: number }`
    * A reference to a neighbor that is still ambiguous.
* **For a cycle**: `{ cycleDistance: number }`
    * A marker used to terminate a recursive expansion when it loops back to an ancestor in the current expansion path. `cycleDistance` is the number of steps back to the first occurrence.

## Core Sorting Logic

The comparison between any two signatures, `sigA` and `sigB`, follows a strict hierarchy of rules. This ensures a stable and deterministic order, meaning the relative order of two unequal signatures will never change in subsequent passes.

The comparison rules are applied in this exact order:

**1. By `neighborCount` (descending)**
* The signature with the higher `neighborCount` always comes first.
* If `neighborCount` values are different, the comparison stops here.

**2. By `finalIndex` (ascending)**
* *This rule is applied only if `neighborCount` is identical.*
* A signature that has a `finalIndex` comes before one that does not.
* If both have a `finalIndex`, the one with the lower value comes first.
* If `finalIndex` values are also identical (for inter-graph comparison), sort by `finishStep` (ascending).

**3. By `cycleDistance` (ascending)**
* *This rule is applied only if the signatures are still tied (same `neighborCount`, neither has a `finalIndex`).*
* A signature with a `cycleDistance` comes before one that does not.
* If both have a `cycleDistance`, the one with the lower value comes first.
* If cycleDistance values are also identical, sort by finishStep (ascending).

**4. By `neighbors` array (recursive)**
* *This rule is applied only if the signatures are still tied.*
* If one signature is `EXPANDED` (has a `neighbors` array) and the other is `COLLAPSED` (does not), the `EXPANDED` one comes first.
* If both are `EXPANDED`, sort by recursively comparing their `neighbors` arrays.
* If both are `COLLAPSED` (i.e., not yet expanded in the current pass), they are considered **equal for now**.

## The Process

The algorithm proceeds in passes, attempting to finalize signatures at each step.

### Initialization

1.  For each node in the graph, create a `Node Signature` object in the `COLLAPSED` state, populating its `label` and `neighborCount`.
2.  Sort the entire list of signatures using the `Core Sorting Logic`.
3.  Scan the sorted list. Any signature that is unique is considered unambiguous. Assign it a `finalIndex` (its current position in the array) and a `finishStep` (which is 1).

### Main Loop (Subsequent Passes)

If, after a pass, there are still ambiguous signatures (those without a `finalIndex`), a new pass is required.

1.  For each ambiguous `Node Signature`, we expand it by populating its `neighbors` array.
2.  For each neighbor of the corresponding node in the graph, we create a `Neighbor Representation` and add it to the array:
    * If the neighbor's signature is already `FINALIZED`, add a `{ finalIndex: ... }` representation.
    * If the neighbor's signature is not finalized, add a `{ neighborCount: ... }` representation.
3.  Once populated, each new `neighbors` array is sorted using the same `Core Sorting Logic`.
4.  After all ambiguous signatures are expanded, the entire list is re-sorted globally.
5.  The algorithm then attempts to assign a `finalIndex` and `finishStep` to any signature that has now become unique.

### Cycle Detection

During the recursive expansion of a signature (e.g., E -> F -> ...), the algorithm must track the current expansion path. If it attempts to expand a neighbor that is already an ancestor in this path (e.g., expanding F's neighbors and re-encountering E), a cycle is detected.

* The expansion of that branch is terminated.
* The neighbor is represented with a `{ cycleDistance: number }` object, marking the loop.

### Termination

The algorithm terminates when either:
* All signatures have been assigned a `finalIndex`.
* A full pass completes with no new signatures being finalized. This occurs when remaining ambiguities are due to perfect structural symmetry, which the algorithm has correctly identified.

### Final Signature Generation

Once the algorithm terminates, the list of `Node Signature` objects can be converted into a cleaner, final format for output. This typically involves removing working properties like `label` to leave a purely structural, canonical signature.

# Running the Signature v2 algorithm
Now we are going to run the algorithm on this graph

![fig1: Simple Graph](images/simple-graph_G1.png)

## Pass 1: Initialization

This first pass establishes the baseline signatures and resolves any nodes that are immediately unique based on their number of neighbors.

---
### 1. Create Initial Signatures
First, we create a `Node Signature` object for each node, containing only its `label` and `neighborCount`.
<pre>
[
    { "label": "A", "neighborCount": 3 },
    { "label": "B", "neighborCount": 4 },
    { "label": "C", "neighborCount": 1 },
    { "label": "D", "neighborCount": 2 },
    { "label": "E", "neighborCount": 2 },
    { "label": "F", "neighborCount": 2 }
]
</pre>

---
### 2. Sort the List
Next, we sort this list using the `Core Sorting Logic`. At this stage, only the first rule applies: sort by `neighborCount` in **descending** order.

<pre>
[
    { "label": "B", "neighborCount": 4 }, // index 0
    { "label": "A", "neighborCount": 3 }, // index 1
    { "label": "D", "neighborCount": 2 }, // index 2
    { "label": "E", "neighborCount": 2 }, // index 3
    { "label": "F", "neighborCount": 2 }, // index 4
    { "label": "C", "neighborCount": 1 }  // index 5
]
</pre>

---
### 3. Finalize Unambiguous Signatures
Now, we scan the sorted list to find signatures that are unique. A signature is unique if no other signature in the list has the same properties (at this point, just the `neighborCount`).

* **Node B** (`neighborCount: 4`) is **unique**.
* **Node A** (`neighborCount: 3`) is **unique**.
* **Nodes D, E, F** (`neighborCount: 2`) are **not unique**. They are tied with each other.
* **Node C** (`neighborCount: 1`) is **unique**.

We assign a `finalIndex` (its current position in the array) and `finishStep: 1` to each unique signature.

---
### Result at the End of Pass 1
The list of signatures is now in the following state. Nodes **B**, **A**, and **C** are considered **`FINALIZED`**. Nodes D, E, and F remain ambiguous.

```json
[
    { "label": "B", "neighborCount": 4, "finalIndex": 0, "finishStep": 1 },
    { "label": "A", "neighborCount": 3, "finalIndex": 1, "finishStep": 1 },
    { "label": "D", "neighborCount": 2 },
    { "label": "E", "neighborCount": 2 },
    { "label": "F", "neighborCount": 2 },
    { "label": "C", "neighborCount": 1, "finalIndex": 5, "finishStep": 1 }
]
</pre>
```

The algorithm must proceed to a second pass to resolve the ambiguity between D, E, and F.

## Pass 2: Resolving Ambiguities

At the start of this pass, signatures for D, E, and F are ambiguous as they share the same `neighborCount`. We will now expand these three signatures.

---
### 1. Expand Ambiguous Signatures

We populate the `neighbors` array for each ambiguous signature (D, E, and F) using the `Neighbor Representation` rules, keeping the `label` for clarity.

* **For Node D (Neighbors: A, B):**
    * Neighbor 'A' is `FINALIZED` with `finalIndex: 1`.
    * Neighbor 'B' is `FINALIZED` with `finalIndex: 0`.
    * D's `neighbors` array becomes: `[ { "label": "A", "finalIndex": 1 }, { "label": "B", "finalIndex": 0 } ]`

* **For Node E (Neighbors: B, F):**
    * Neighbor 'B' is `FINALIZED` with `finalIndex: 0`.
    * Neighbor 'F' is **not** finalized. Its `neighborCount` is 2.
    * E's `neighbors` array becomes: `[ { "label": "B", "finalIndex": 0 }, { "label": "F", "neighborCount": 2 } ]`

* **For Node F (Neighbors: B, E):**
    * Neighbor 'B' is `FINALIZED` with `finalIndex: 0`.
    * Neighbor 'E' is **not** finalized. Its `neighborCount` is 2.
    * F's `neighbors` array becomes: `[ { "label": "B", "finalIndex": 0 }, { "label": "E", "neighborCount": 2 } ]`

---
### 2. Sort Internal `neighbors` Arrays

Next, we sort each of the newly created `neighbors` arrays using the `Core Sorting Logic`. The `label` is not used for sorting.

* **D's sorted `neighbors`**: `[ { "label": "B", "finalIndex": 0 }, { "label": "A", "finalIndex": 1 } ]`
* **E's sorted `neighbors`**: `[ { "label": "B", "finalIndex": 0 }, { "label": "F", "neighborCount": 2 } ]` (already sorted)
* **F's sorted `neighbors`**: `[ { "label": "B", "finalIndex": 0 }, { "label": "E", "neighborCount": 2 } ]` (already sorted)

---
### 3. Re-sort the Global List & Finalize

With the ambiguous signatures now expanded, we re-sort the entire list of six signatures. The sorting logic compares D, E, and F based on their new `neighbors` arrays.

* The signature for **D** is now unique.
* The signatures for **E** and **F** are identical to each other from a sorting perspective (since `label` is ignored in the comparison), so they remain ambiguous.

Node D's unique signature earns it a `finalIndex` of 2 (its new stable position in the sorted list) and a `finishStep` of 2.

---
### Result at the End of Pass 2

The list of signatures is now in the following state. Node **D** is now `FINALIZED`. Nodes E and F remain ambiguous.

```json
[
  { "label": "B", "neighborCount": 4, "finalIndex": 0, "finishStep": 1 },
  { "label": "A", "neighborCount": 3, "finalIndex": 1, "finishStep": 1 },
  { "label": "D", "neighborCount": 2, "neighbors": [ { "label": "B", "finalIndex": 0 }, { "label": "A", "finalIndex": 1 } ], "finalIndex": 2, "finishStep": 2 },
  { "label": "E", "neighborCount": 2, "neighbors": [ { "label": "B", "finalIndex": 0 }, { "label": "F", "neighborCount": 2 } ] },
  { "label": "F", "neighborCount": 2, "neighbors": [ { "label": "B", "finalIndex": 0 }, { "label": "E", "neighborCount": 2 } ] },
  { "label": "C", "neighborCount": 1, "finalIndex": 5, "finishStep": 1 }
]

## Pass 3: Deep Expansion and Termination

At the start of this pass, nodes E and F are still ambiguous, with identical signatures. We must expand their signatures further by looking inside their `neighbors` arrays for unresolved parts.

---
### 1. Recursive Expansion of Signatures

The current signature for both E and F contains a `Neighbor Representation` of an unresolved node: `{ "label": "F", "neighborCount": 2 }` for E, and `{ "label": "E", "neighborCount": 2 }` for F. We will now expand this part for each.

* **Expanding E's Signature:**
    * We expand the representation of node **F**.
    * The expansion path is now **E -> F**.
    * We look at F's actual neighbors: **B** and **E**.
    * Neighbor 'B' is `FINALIZED`. Its representation is `{ "label": "B", "finalIndex": 0 }`.
    * Neighbor 'E' is the root of our current expansion path, creating a **cycle**. The path is `E -> F -> E` (distance 2). Since this is Pass 3, the representation is **`{ "label": "E", "cycleDistance": 2, "finishStep": 3 }`**.
    * The fully expanded representation for the neighbor F becomes `{ "label": "F", "neighborCount": 2, "neighbors": [ { "label": "B", "finalIndex": 0 }, { "label": "E", "cycleDistance": 2, "finishStep": 3 } ] }`.
    * E's full signature is updated accordingly.

* **Expanding F's Signature:**
    * The process is perfectly symmetrical. We expand its representation of node **E**.
    * The expansion path is **F -> E**.
    * E's neighbors are **B** and **F**.
    * Neighbor 'B' is `FINALIZED` (`{ "label": "B", "finalIndex": 0 }`).
    * Neighbor 'F' is the root of this path (**F -> E -> F**), creating a cycle. The representation is **`{ "label": "F", "cycleDistance": 2, "finishStep": 3 }`**.
    * The fully expanded representation for the neighbor E becomes `{ "label": "E", "neighborCount": 2, "neighbors": [ { "label": "B", "finalIndex": 0 }, { "label": "F", "cycleDistance": 2, "finishStep": 3 } ] }`.
    * F's full signature is updated.

---
### 2. Final Comparison and Termination

After the deep expansion, we compare the new, complex signatures of E and F. They are still structurally identical from a sorting perspective, as the labels 'E' and 'F' within their cycle representations are not used for comparison.

The signatures are **still considered identical by the sorter**.

Because no new signatures were finalized in this pass, the algorithm's termination condition is met. The process is complete.

---
### Result at the End of Pass 3 (Final Working Signatures)
The final state of the working signatures before the cleanup step is:

```json
[
  { "label": "B", "neighborCount": 4, "finalIndex": 0, "finishStep": 1 },
  { "label": "A", "neighborCount": 3, "finalIndex": 1, "finishStep": 1 },
  { "label": "D", "neighborCount": 2, "neighbors": [ { "label": "B", "finalIndex": 0 }, { "label": "A", "finalIndex": 1 } ], "finalIndex": 2, "finishStep": 2 },
  { "label": "E", "neighborCount": 2, "neighbors": [ { "label": "B", "finalIndex": 0 }, { "label": "F", "neighborCount": 2, "neighbors": [ { "label": "B", "finalIndex": 0 }, { "label": "E", "cycleDistance": 2, "finishStep": 3 } ] } ] },
  { "label": "F", "neighborCount": 2, "neighbors": [ { "label": "B", "finalIndex": 0 }, { "label": "E", "neighborCount": 2, "neighbors": [ { "label": "B", "finalIndex": 0 }, { "label": "F", "cycleDistance": 2, "finishStep": 3 } ] } ] },
  { "label": "C", "neighborCount": 1, "finalIndex": 5, "finishStep": 1 }
]
```

# Final Step : Remove Labels

The last step consist only of removing the labels.

```json
[
  { "neighborCount": 4, "finalIndex": 0, "finishStep": 1 },
  { "neighborCount": 3, "finalIndex": 1, "finishStep": 1 },
  { "neighborCount": 2, "neighbors": [ { "finalIndex": 0 }, { "finalIndex": 1 } ], "finalIndex": 2, "finishStep": 2 },
  { "neighborCount": 2, "neighbors": [ { "finalIndex": 0 }, { "neighborCount": 2, "neighbors": [ { "finalIndex": 0 }, { "cycleDistance": 2, "finishStep": 3 } ] } ] },
  { "neighborCount": 2, "neighbors": [ { "finalIndex": 0 }, { "neighborCount": 2, "neighbors": [ { "finalIndex": 0 }, { "cycleDistance": 2, "finishStep": 3 } ] } ] },
  { "neighborCount": 1, "finalIndex": 5, "finishStep": 1 }
]
```


# Interpreting the Signature Results

## What the Signatures Reveal
This final array is the canonical "fingerprint" of the graph's topology.

### 1. Structural Equivalence (Symmetry)

The most significant conclusion is that **nodes E and F have perfectly identical signatures.** This means the algorithm has proven that from a topological standpoint, nodes E and F are indistinguishable and occupy symmetrical, interchangeable positions in the graph. They belong to the same "orbit" of the graph's automorphism group.

### 2. Unique Structural Roles

The algorithm successfully assigned a unique signature and `finalIndex` to four of the six nodes: **A, B, C, and D**. This demonstrates that these four nodes each have a distinct structural role within the graph that can be differentiated by their connectivity patterns.

### 3. A Hierarchy of Complexity

The `finishStep` value in each signature reveals how "difficult" it was for the algorithm to resolve each node's role, creating a structural hierarchy:

* **Simplest Roles (`finishStep: 1`):** Nodes **B, A, and C** were identified immediately, defined purely by their number of neighbors.
* **Intermediate Role (`finishStep: 2`):** Node **D** required looking at its immediate neighbors to be resolved.
* **Most Complex Roles (`finishStep: 3`):** Nodes **E and F** are the most complex, defined by a recursive, cyclical relationship that required a 3-pass analysis.

### 4. A Canonical Graph Signature

This final, sorted array of label-less signatures is a **canonical representation of the entire graph**. This output can be used for comparison; if another graph produces the exact same final array, the two graphs are **isomorphic** (structurally identical).

## Limitations and Considerations

While this algorithm produces a highly detailed and deterministic signature, it's important to acknowledge two significant trade-offs: its computational complexity and the potential size of the generated signatures.
### A Note on Empirical Validation

It is important to state that the algorithm detailed here is a formal specification. While its logic was battle-tested and refined using our 6-node example, it has **not yet been implemented in code or benchmarked** against a wide variety of complex graphs.

Therefore, its real-world performance characteristics, memory consumption, and correctness on edge cases are still theoretical. The logical next step would be a robust implementation to validate these results empirically and confirm its practical effectiveness.

### Computational Complexity

The performance of this algorithm is not its primary strength. The complexity can be considerable, especially for large or densely connected graphs. The main drivers of this complexity are:

1.  **Sorting in Each Pass:** The need to re-sort the entire list of node signatures after each expansion pass is computationally intensive.
2.  **Recursive Comparison:** The core comparison logic, which may involve deep, lexicographical comparisons of nested `neighbors` arrays, can be costly for nodes in complex regions of the graph.

This algorithm is therefore better suited for offline, in-depth structural analysis where descriptive accuracy is paramount, rather than for high-performance or real-time applications.

### Signature Size

The canonical signatures themselves, while rich in information, can become very large and deeply nested. This is because the signature of a node effectively encodes the structure of its surrounding neighborhood, potentially to a great depth, to resolve ambiguities.

Consequently, the storage requirements for the final set of signatures for a large graph can be substantial.

Ultimately, this algorithm trades performance and conciseness for an extremely high degree of structural detail and accuracy.

# Conclusion

Now that the blueprint of the algorithm is complete, the ultimate question remains: does this theoretical power survive the harsh realities of implementation?
Signature v1 was an 'order 3' algorithm. Now, with a recursive engine built to handle arbitrary depth, we must embark on the next phase to measure the true, and perhaps unlimited, order of our Signature V2.